In [1]:
TRAIN_PATH = "Dataset/Train"
VAL_PATH = "Dataset/validations"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

Using TensorFlow backend.


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [7]:
def conv_block(ni, size=3,stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return Sequential(
        [
        Conv2D(ni, kernel_size=size, padding="same",use_bias=False), 
        BatchNormalization(),
        LeakyReLU(alpha=0.1)  
        ]
    )

In [8]:
def triple_conv(ni,nf):
    return Sequential(
        [
        conv_block(nf),
        conv_block(ni,size=1),  
        conv_block(nf)
        ]
    )

In [9]:
def maxpooling():
    return MaxPooling2D(2, strides=2)

In [71]:
def inital():
    return Sequential(
        [
        Conv2D(8,kernel_size=(3,3),padding="same",input_shape=(224,224,3)),
        BatchNormalization(),
        LeakyReLU(alpha=0.1)  
        ]
    )

In [1]:
#CNN based model

model = Sequential(
    [
    inital(), 
    maxpooling(),
    conv_block(16),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32,64),
    maxpooling(),
    triple_conv(64,128),
    maxpooling(),
    triple_conv(128,256),
    conv_block(128,size=1),
    conv_block(256),
    Flatten(),
    Dense(256,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
    ]
)

model.compile(loss=keras.losses.binary_crossentropy, optimizer=Adam(lr=0.0005), metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [64]:
model.summary()

Model: "sequential_238"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_182 (Conv2D)          (None, 224, 224, 8)       224       
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 112, 112, 8)       0         
_________________________________________________________________
sequential_219 (Sequential)  (None, 112, 112, 16)      1216      
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 56, 56, 16)        0         
_________________________________________________________________
sequential_223 (Sequential)  (None, 56, 56, 32)        10048     
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 28, 28, 32)        0         
_________________________________________________________________
sequential_227 (Sequential)  (None, 28, 28, 64)     

In [65]:
train_data = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset = image.ImageDataGenerator(rescale = 1./255)

In [66]:
train_generator = train_data.flow_from_directory(
    'Dataset/Train',
    target_size = (224,224),
    batch_size= 32,
    class_mode = 'binary',
)

Found 304 images belonging to 2 classes.


In [67]:
train_generator.class_indices

{'COVID': 0, 'Normal': 1}

In [68]:
validation_generator = test_dataset.flow_from_directory(
    'Dataset/validations',
    target_size = (224,224),
    batch_size= 32,
    class_mode = 'binary',
)

Found 76 images belonging to 2 classes.


In [69]:
hist = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=20,
    validation_data = validation_generator,
    validation_steps=2,
)

Epoch 1/20
8/8 [==============================] - 10s 1s/step - loss: 0.6960 - accuracy: 0.7667 - val_loss: 0.7068 - val_accuracy: 0.4844
Epoch 2/20
8/8 [==============================] - 10s 1s/step - loss: 0.2055 - accuracy: 0.9625 - val_loss: 0.7604 - val_accuracy: 0.4844
Epoch 3/20
8/8 [==============================] - 10s 1s/step - loss: 0.1866 - accuracy: 0.9458 - val_loss: 0.8586 - val_accuracy: 0.4688
Epoch 4/20
8/8 [==============================] - 11s 1s/step - loss: 0.0826 - accuracy: 0.9648 - val_loss: 1.2546 - val_accuracy: 0.4688
Epoch 5/20
8/8 [==============================] - 10s 1s/step - loss: 0.1656 - accuracy: 0.9458 - val_loss: 1.8327 - val_accuracy: 0.4844
Epoch 6/20
8/8 [==============================] - 11s 1s/step - loss: 0.1232 - accuracy: 0.9833 - val_loss: 2.6918 - val_accuracy: 0.5000
Epoch 7/20
8/8 [==============================] - 12s 1s/step - loss: 0.2163 - accuracy: 0.9453 - val_loss: 3.1619 - val_accuracy: 0.4688
Epoch 8/20
8/8 [==================